In [ ]:
#Importing Necessary Libraries 
import pandas as pd
import numpy as np

In [26]:
# Reducing to necessary columns and year 2020 
df = pd.read_csv('../data/all_songs_data_processed.csv')
data20 = df[df['Year'] == 2020]
data20 = data20[['Artist', 'Song Title', 'Lyrics', 'Corpus']]
data20.head()

,Artist,Song Title,Lyrics,Corpus
5913,The Weeknd,Blinding Lights,Yeah I've been tryna call I've been on my ow...,yeah tryna long maybe love maybe withdrawal tu...
5914,Post Malone,Circles,"Oh, oh, oh Oh, oh, oh Oh, oh, oh, oh, oh W...",oh oh oh oh oh oh oh oh oh oh oh turn til upsi...
5915,Roddy Ricch,The Box,Pullin' out the coupe at the lot Told 'em fuck...,Pullin coupe lot tell fuck fuck SWAT Bustin be...
5916,Dua Lipa,Don't Start Now,If you don't wanna see me Did a full one-eig...,wanna eighty crazy thinking way heartbreak cha...
5917,Harry Styles,Adore You,Walk in your rainbow paradise (Paradise) Straw...,walk rainbow paradise Paradise Strawberry lips...


In [9]:
pip install bertopic sentence-transformers

  Obtaining dependency information for bertopic from https://files.pythonhosted.org/packages/98/05/2d6b305391efff89c2b4cf19cf847f971ca163eb5c149d0d2ffac0a9c7ed/bertopic-0.17.3-py3-none-any.whl.metadata
  Obtaining dependency information for sentence-transformers from https://files.pythonhosted.org/packages/48/21/4670d03ab8587b0ab6f7d5fa02a95c3dd6b1f39d0e40e508870201f3d76c/sentence_transformers-5.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for hdbscan>=0.8.29 from https://files.pythonhosted.org/packages/26/6b/88b8c8023c0c0b27589ad83c82084a1b751917a3e09bdf7fcacf7e6bd523/hdbscan-0.8.40-cp311-cp311-macosx_10_9_universal2.whl.metadata
  Obtaining dependency information for umap-learn>=0.5.0 from https://files.pythonhosted.org/packages/6b/b1/c24deeda9baf1fd491aaad941ed89e0fed6c583a117fd7b79e0a33a1e6c0/umap_learn-0.5.9.post2-py3-none-any.whl.metadata
  Obtaining dependency information for transformers<5.0.0,>=4.41.0 from https://files.pythonhosted.org/packages/e5/2b/4d270

In [ ]:
df = data20
lyrics = df["Lyrics"].tolist()

In [11]:
from bertopic import BERTopic

In [13]:
# Filter out non-string lyrics (e.g., nan values)
clean_lyrics = [lyric if isinstance(lyric, str) else "" for lyric in lyrics]

topic_model = BERTopic(language="english")
topics, probs = topic_model.fit_transform(clean_lyrics)

In [14]:
topic_model.get_topic_info().head()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,15,-1_the_featuring_la_me,"[the, featuring, la, me, que, to, of, de, no, ...",[Beyoncé COLORS Black Pumas ROCKSTAR DaBaby ...
1,0,51,0_you_the_to_me,"[you, the, to, me, my, and, love, it, dont, that]",[There stands the glass That will ease all my ...
2,1,34,1_the_you_it_im,"[the, you, it, im, in, and, yeah, to, my, that]","[Ugh, you're a monster [Verse 1: Eminem] I ca..."


In [15]:
topic_model.get_topic(0)

[('you', 0.11878008742688179),
 ('the', 0.07099698147244424),
 ('to', 0.061392393863085926),
 ('me', 0.05876725550909064),
 ('my', 0.05522073973419982),
 ('and', 0.0513015323466842),
 ('love', 0.04819094692700823),
 ('it', 0.04757841375944383),
 ('dont', 0.045781082354474076),
 ('that', 0.040173285300268854)]